<a href="https://colab.research.google.com/github/LxYuan0420/eat_tensorflow2_in_30_days/blob/master/notebooks/6_5_Model_Training_Using_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

It only requires six additional lines of code when training your model using TPU on Google Colab.

In Colab notebook, choose TPU in Edit -> Notebook Settings -> Hardware Accelerator.

Note: the following code only executes on Colab.

You may use the following link for testing (tf_TPU, in Chinese)

https://colab.research.google.com/drive/1XCIhATyE1R7lq6uwFlYlRsUr5d9_-r1s

In [1]:
import tensorflow as tf
tf.__version__

'2.4.1'

In [2]:
from tensorflow.keras import *

**1. Data Preparation**

In [13]:
MAX_LEN = 300
BATCH_SIZE = 8 #reduce from 32 to 8, got error: Unable to parse tensor proto

(x_train, y_train), (x_test, y_test) = datasets.reuters.load_data()
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=MAX_LEN)
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=MAX_LEN)

MAX_WORDS = x_train.max() + 1
CAT_NUM = y_train.max() + 1

ds_train = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE).cache()
ds_test = tf.data.Dataset.from_tensor_slices((x_test, y_test)).shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE).cache()

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/datasets/reuters.py:148: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/datasets/reuters.py:149: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [14]:
def create_model():

    model = models.Sequential()
    model.add(layers.Embedding(MAX_WORDS, 7, input_length=MAX_LEN))
    model.add(layers.Conv1D(filters=64, kernel_size=5, activation='relu'))
    model.add(layers.MaxPool1D())
    model.add(layers.Conv1D(filters=32, kernel_size=3, activation='relu'))
    model.add(layers.MaxPool1D())
    model.add(layers.Flatten())
    model.add(layers.Dense(CAT_NUM, activation='softmax'))

    return model

def compile_model(model):
    model.compile(
        loss = losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer = optimizers.Nadam(),
        metrics = [metrics.SparseCategoricalAccuracy(), metrics.SparseTopKCategoricalAccuracy()]
    )

    return model

**3. Model Training**

In [15]:
# the 6 additional lines of code
import os
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu="grpc://" + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

strategy = tf.distribute.experimental.TPUStrategy(resolver)
with strategy.scope():
    model = create_model()
    model.summary()
    model = compile_model(model)


INFO:tensorflow:Initializing the TPU system: grpc://10.105.197.218:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.105.197.218:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 300, 7)            216874    
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 296, 64)           2304      
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 148, 64)           0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 146, 32)           6176      
_________________________________________________________________
max_pooling1d_9 (MaxPooling1 (None, 73, 32)            0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 2336)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 46)               

In [16]:
history = model.fit(ds_train, validation_data=ds_test, epochs=10)

Epoch 1/10
1123/1123 [==============================] - 34s 26ms/step - loss: 2.1022 - sparse_categorical_accuracy: 0.4564 - sparse_top_k_categorical_accuracy: 0.7294 - val_loss: 1.5396 - val_sparse_categorical_accuracy: 0.6113 - val_sparse_top_k_categorical_accuracy: 0.7760
Epoch 2/10
1123/1123 [==============================] - 28s 25ms/step - loss: 1.3959 - sparse_categorical_accuracy: 0.6370 - sparse_top_k_categorical_accuracy: 0.8020 - val_loss: 1.4756 - val_sparse_categorical_accuracy: 0.6402 - val_sparse_top_k_categorical_accuracy: 0.8085
Epoch 3/10
1123/1123 [==============================] - 28s 25ms/step - loss: 1.0306 - sparse_categorical_accuracy: 0.7242 - sparse_top_k_categorical_accuracy: 0.8867 - val_loss: 1.6521 - val_sparse_categorical_accuracy: 0.6349 - val_sparse_top_k_categorical_accuracy: 0.8094
Epoch 4/10
1123/1123 [==============================] - 28s 25ms/step - loss: 0.6970 - sparse_categorical_accuracy: 0.8202 - sparse_top_k_categorical_accuracy: 0.9459 - val